In [8]:
from gameanalysis import AGGvalidation as AGGV
import os
import shutil
import numpy as np
from subprocess import call
import matplotlib.pyplot as plt
import json
machines = ['parsley', 'cocoa', 'squash', 'cinnamon', 'gale', 'cucumber', 'vinegar', 'chicory']
experiment_name = 'Noise1'

In [9]:
# Create an empty directory to store the experiment input, script, and output files
# Caution: Will empty an existing directory
directory = '/scratch/fyang1/GameAnalysis/parallel/{}'.format(experiment_name)
if not os.path.exists(directory):
    os.makedirs(directory)
else:
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        os.remove(file_path)
shutil.copy('/scratch/fyang1/GameAnalysis/parallel/ex.py', directory)

'/scratch/fyang1/GameAnalysis/parallel/Noise1/ex.py'

In [10]:
def create_work(directory, worklist, num_machines):
    nums = np.ones(num_machines, dtype=int) * int(len(worklist) / num_machines)
    nums[:len(worklist) % num_machines] += 1
    nums = np.cumsum(nums)
    works = np.split(worklist, nums)

    for i in range(num_machines):
        infile_name = os.path.join(directory, '{}.in'.format(i))
        f = open(infile_name, 'w')
        for params in works[i]:
            f.write(str(list(params))+'\n')
        wkfile_name = os.path.join(directory, '{}.wk'.format(i))
        f = open(wkfile_name, 'w')
        f.write('cd {}\n. /scratch/fyang1/GameAnalysis/bin/activate\npython3 ex.py {}'.format(directory, i))

In [11]:
def create_script(directory, machines):
    for i, machine in enumerate(machines):
        f = open(os.path.join(directory, '{}.sh'.format(i)), 'w')
        f.write('#!/usr/bin/expect -f\nset timeout -1\nspawn ssh fyang1@{}\nexpect "\\$ "\nsend "{}/{}.wk\\r"\nexpect "\\$ "\nsend "exit\\r"'.format(machine, directory, i))

In [12]:
def analyze(directory, num_machines):
    results = []
    for i in range(num_machines):
        f = open(os.path.join(directory, '{}.out'.format(i)))
        for line in f:
            results.append(eval(line))
    return np.array(results).T

In [13]:
def record_results(ret, worklist, name):
    ret_T = ret.T
    results = []
    for i, p in enumerate(worklist):
        results.append((dict(zip(['num_players', 'num_facilities', 'num_required', \
                                  'proportion', 'noise', 'samples', 'repetitions'], p)),\
                        dict(zip(['MAE', 'MAPE', 'SMAPE', 'RS_gap', 'AGG_gap'], ret_T[i]))))
    f = open('/scratch/fyang1/GameAnalysis/results/{}'.format(name), 'w')
    f.write(json.dumps(results))

In [14]:
worklist = AGGV.noise_experiment([6,5,2], num_reps=50)
print(worklist)
create_work(directory, worklist, len(machines))
create_script(directory, machines)

[[6 5 2 1 (0, 0.0) 1 50]
 [6 5 2 1 (0, 0.25) 1 50]
 [6 5 2 1 (0, 0.5) 1 50]
 [6 5 2 1 (0, 0.75) 1 50]
 [6 5 2 1 (0, 1.0) 1 50]
 [6 5 2 1 (0, 1.25) 1 50]
 [6 5 2 1 (0, 1.5) 1 50]
 [6 5 2 1 (0, 1.75) 1 50]
 [6 5 2 1 (0, 2.0) 1 50]
 [6 5 2 1 (0, 2.25) 1 50]
 [6 5 2 1 (0, 2.5) 1 50]
 [6 5 2 1 (0, 2.75) 1 50]
 [6 5 2 1 (0, 3.0) 1 50]
 [6 5 2 1 (0, 3.25) 1 50]
 [6 5 2 1 (0, 3.5) 1 50]
 [6 5 2 1 (0, 3.75) 1 50]
 [6 5 2 1 (0, 4.0) 1 50]
 [6 5 2 1 (0, 4.25) 1 50]
 [6 5 2 1 (0, 4.5) 1 50]
 [6 5 2 1 (0, 4.75) 1 50]]


In [ ]:
# To use the scripts:
# 1. ssh-add, 
# 2. chmod 733 *.sh *.wk 
# 3. for i in *.sh;do; ./$i &; done (run all the .sh files background)

In [7]:
ret = analyze(directory, len(machines))

In [28]:
j = record_results(ret, worklist, experiment_name)

In [11]:
plt.title('GP Pure Strategy Payoff Prediction Accuracy')
plt.xlabel('Percentage of Profiles in Original Game used in GP Training (%)')
plt.ylabel('Error')
l1, = plt.plot(np.arange(0.05, 1.05, 0.05), ret[0])
l2, = plt.plot(np.arange(0.05, 1.05, 0.05), ret[1])
l3, = plt.plot(np.arange(0.05, 1.05, 0.05), ret[2])
plt.legend([l1, l2, l3], ['MAE', 'MAPE', 'SMAPE'])
plt.show()